In [1]:
import collections
import numpy as np
from operator import add
from functools import reduce
from bio_spark.io.fasta_reader import FASTAReader
from pyspark.sql import Row, SparkSession, functions
from pyspark.context import SparkContext, SparkConf

In [27]:
def seq2kmer(seq_):
    value = seq_[0].strip()    
    k = 3
    num_kmers = len(value) - k + 1
    kmers_list = [value[n*k:k*(n+1)] for n in range(0, num_kmers)]
    
    return kmers_list


seq2kmer_udf = functions.udf(seq2kmer)


def kmers_list2kmers_freq_dict(kmers_list):
    kmers_list_np = np.array(kmers_list)
    flatten_data = kmers_list_np.flatten()
    unique, counts = np.unique(flatten_data, return_counts=True)
    kmers_list = list(zip(unique.tolist(), counts.tolist()))
    return kmers_list


kmers_list2kmers_freq_dict_udf = functions.udf(kmers_list2kmers_freq_dict)

In [3]:
sConf = SparkConf("spark://localhost:7077")

In [4]:
sc = SparkContext(conf=sConf)
spark = SparkSession(sc)

In [8]:
!ls ../data/genomes

Prochlorococcus_sp_W10_genomic.fna Prochlorococcus_sp_W4_genomic.fna
Prochlorococcus_sp_W11_genomic.fna Prochlorococcus_sp_W6_genomic.fna
Prochlorococcus_sp_W12_genomic.fna Prochlorococcus_sp_W7_genomic.fna
Prochlorococcus_sp_W2_genomic.fna  Prochlorococcus_sp_W8_genomic.fna
Prochlorococcus_sp_W3_genomic.fna  Prochlorococcus_sp_W9_genomic.fna


In [15]:
plain_df = sc.textFile(
    "../data/genomes/Prochlorococcus_sp_W10_genomic.fna"
).map(lambda x: Row(row=x)).toDF()

In [16]:
reader = FASTAReader(sc)

In [24]:
parsedDF = reader.read(plain_df, "fasta")

In [25]:
parsedDF.coalesce(2)

DataFrame[seqID: struct<row:string>, seq: struct<row:string>]

In [28]:
kmers_of_seqs_df = parsedDF.withColumn("kmers", seq2kmer_udf("seq"))

In [31]:
#print(kmers_of_seqs_df.show())

grouped_by_seq_df = kmers_of_seqs_df.groupby("seqID")\
                        .agg(functions.collect_list('kmers')\
                        .alias('kmers_list'))\
                        .withColumn('kmers_freq',
                        kmers_list2kmers_req_dict_udf('kmers_list')
                                   )

NameError: name 'kmers_list2kmers_req_dict_udf' is not defined